### Contents

- [Header](#Header)
- [Import Data](#Import-Data)
- [Functions](#Functions)


- [Pre-process Data](#Pre-process-Data)
- [Plot df_model](#Plot-df_model)


- [Create Features and Target](#Create-Features-and-Target)
- [Handle Imbalanced Data](#Handle-Imbalanced-Data)


- [Logistic Regresion Model](#Logistic-Regresion-Model)
- [KNN Model](#KNN-Model)
- [DTC Model](#DTC-Model)
- [RTC Model](#RTC-Model)
- [GridSearch Model](#GridSearch-Model)

### Header

In [1]:
# import libraries

# maths
import numpy as np
import pandas as pd

# visual
#from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pydotplus

# modelling
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix,accuracy_score,r2_score,mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.utils import resample, shuffle
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.externals.six import StringIO
from imblearn.over_sampling import SMOTE

# Others
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# file paths

raw_path = '../../data/0_raw/fitrec/' 
input_path = '../../data/1_input/fitrec/'
clean_path = '../../data/2_clean/fitrec/' 
preprocess_path = '../../data/3_preprocess/fitrec/' 
output_path = '../../data/4_output/fitrec/'

sports_path = '../../data/1_input/sports/' 

### Import Data

In [3]:
# import data

#file = 'endomondoHR_proper_summary.csv'
file = 'endomondoHR_proper_dist_spd_summary.csv'

in_path = clean_path + file

df = pd.read_csv(in_path)

In [4]:
# import sports.xlsx

path = sports_path + 'sports.xlsx'
df_sports = pd.read_excel(path)
df_sports

sport             sport_rename       type  \
0                  aerobics                 aerobics        gym   
1                 badminton                badminton     racket   
2                basketball               basketball       team   
3                      bike                     bike    aerobic   
4          bike (transport)                     bike    aerobic   
5            indoor cycling                     bike    aerobic   
6             mountain bike                     bike    aerobic   
7          circuit training         circuit training    aerobic   
8                  climbing                 climbing    aerobic   
9   core stability training  core stability training     indoor   
10               elliptical               elliptical        gym   
11                     golf                     golf  gentlemen   
12               gymnastics               gymnastics     indoor   
13                   hiking                   hiking    aerobic   
14         horseback riding         horseback riding  gentlemen   
15                 kayaking                 kayaking      water   
16             martial arts             martial arts     indoor   
17             orienteering             orienteering    aerobic   
18                   rowing            rowing             water   
19                    rugby                    rugby       team   
20                      run                      run    aerobic   
21        treadmill running                      run    aerobic   
22                  sailing                  sailing      water   
23                    skate                    skate    aerobic   
24     cross-country skiing                   skiing     winter   
25          downhill skiing                   skiing     winter   
26            roller skiing                   skiing     winter   
27             snowboarding             snowboarding     winter   
28              snowshoeing              snowshoeing     winter   
29                   soccer                   soccer       team   
30          squash                            squash     racket   
31          stair climing             stair climbing    aerobic   
32             kite surfing                  surfing      water   
33              windsurfing                  surfing      water   
34                 swimming                 swimming      water   
35             table tennis             table tennis     racket   
36                   tennis                   tennis     racket   
37          fitness walking                     walk    aerobic   
38        treadmill walking                     walk    aerobic   
39                     walk                     walk    aerobic   
40          weight training          weight training        gym   
41                  pilates                     yoga     indoor   
42                     yoga                     yoga     indoor   

             venue  location_valid  distance_valid  speed_valid  speed_max  
0           indoor               0               0            0          0  
1           indoor               0               1            0          0  
2   outdoor/indoor               1               1            0          0  
3          outdoor               1               1            1        244  
4          outdoor               1               1            1        244  
5           indoor               0               1            1        244  
6          outdoor               1               1            1        244  
7   outdoor/indoor               0               0            0          0  
8   outdoor/indoor               1               0            0          0  
9           indoor               0               0            0          0  
10          indoor               0               0            0          0  
11         outdoor               1               1            0          0  
12          indoor               0               1            0         

### Functions

### Pre-process Data

In [5]:
df.columns

Index(['id', 'userId', 'gender', 'sport', 'time_start', 'time_end', 'time_dur',
       'lat_start', 'lon_start', 'lat_end', 'lon_end', 'alt_avg', 'alt_min',
       'alt_05', 'alt_25', 'alt_75', 'alt_95', 'alt_max', 'hr_avg', 'hr_min',
       'hr_05', 'hr_25', 'hr_75', 'hr_95', 'hr_max', 'hr_outof', 'hr_fatburn',
       'hr_cardio', 'hr_peak', 'spd_avg', 'spd_min', 'spd_05', 'spd_25',
       'spd_75', 'spd_95', 'spd_max', 'impute'],
      dtype='object')

In [6]:
# remove rows with abnormal heartrate

print(len(df))

cond_1 = df['hr_avg'] > 60
#cond_2 = df['hr_min'] > 0
#cond_3 = df['hr_25'] > 0
#cond_4 = df['hr_75'] > 0
#cond_5 = df['hr_max'] > 0

df = df[cond_1]
#df = df[cond_1 & cond_2 & cond_3 & cond_4 & cond_5]

print(len(df))

167783
167256


In [7]:
# drop rows if speed is nan

# print(len(df))
# df.dropna(subset=['spd_avg'],inplace=True)
# print(len(df))

In [8]:
df['sport'].value_counts()

bike                       71658
run                        70477
mountain bike              10713
bike (transport)            7676
indoor cycling              1725
walk                        1246
orienteering                 867
cross-country skiing         788
core stability training      443
fitness walking              292
skate                        260
roller skiing                238
hiking                       237
kayaking                      92
circuit training              89
weight training               74
rowing                        71
gymnastics                    66
soccer                        51
downhill skiing               42
treadmill running             28
snowshoeing                   16
swimming                      14
golf                          12
badminton                     10
horseback riding              10
elliptical                    10
basketball                     8
tennis                         8
aerobics                       7
climbing  

In [9]:
# select only sports with speed_valid = 1

valid_mask = df_sports['speed_valid'] == 1
valid_sport_list = df_sports[valid_mask]['sport']
valid_sport_list = list(valid_sport_list)

# overwrite: compare specific sports
#valid_sport_list = ['kayaking','rowing']

valid_mask_2 = df['sport'].isin(valid_sport_list)
df = df[valid_mask_2]
df['sport'].unique()

array(['bike', 'bike (transport)', 'run', 'mountain bike', 'rowing',
       'orienteering', 'kayaking', 'indoor cycling', 'skate',
       'cross-country skiing', 'walk', 'hiking', 'treadmill running',
       'swimming', 'snowshoeing', 'snowboarding', 'fitness walking',
       'roller skiing', 'horseback riding', 'downhill skiing',
       'treadmill walking', 'sailing', 'kite surfing', 'windsurfing'],
      dtype=object)

In [10]:
# merge similar sports

for idx,row in df_sports.iterrows():
    
    sport = row['sport'].rstrip()
    sport_rename = row['sport_rename']
    print(sport,sport_rename)
    
    df['sport'].replace(sport,sport_rename,inplace=True)

aerobics aerobics
badminton badminton
basketball basketball
bike bike
bike (transport) bike
indoor cycling bike
mountain bike bike
circuit training circuit training
climbing climbing
core stability training core stability training
elliptical elliptical
golf golf
gymnastics gymnastics
hiking hiking
horseback riding horseback riding
kayaking kayaking
martial arts martial arts
orienteering orienteering
rowing rowing       
rugby rugby
run run
treadmill running run
sailing sailing
skate skate
cross-country skiing skiing
downhill skiing skiing
roller skiing skiing
snowboarding snowboarding
snowshoeing snowshoeing
soccer soccer
squash squash
stair climing stair climbing
kite surfing surfing
windsurfing surfing
swimming swimming
table tennis table tennis
tennis tennis
fitness walking walk
treadmill walking walk
walk walk
weight training weight training
pilates yoga
yoga yoga


In [11]:
# merge similar sports

# df['sport'].replace('treadmill running','run',inplace=True)
# df['sport'].replace(['treadmill walking','fitness walking'],'walk',inplace=True)
# df['sport'].replace(['bike','mountain bike','bike (transport)','indoor cycling'],'cycle',inplace=True)
# df['sport'].replace(['windsurfing','kite surfing'],'surfing',inplace=True)
# df['sport'].replace(['cross-country skiing','downhill skiing','roller skiing'],'skiing',inplace=True)
# df['sport'].replace('pilates','yoga',inplace=True)

In [12]:
df['sport'].value_counts()

bike                91772
run                 70505
walk                 1539
skiing               1068
orienteering          867
skate                 260
hiking                237
kayaking               92
rowing                 71
snowshoeing            16
swimming               14
horseback riding       10
snowboarding            3
surfing                 2
sailing                 1
Name: sport, dtype: int64

In [13]:
# select columns for feature selection

# cols = ['sport', 'time_dur', 'alt_avg', 'alt_min', 'alt_25', 'alt_75','alt_max', 'hr_avg', 'hr_min', 'hr_25', 'hr_75', 'hr_max']

#cols = ['sport','hr_avg', 'hr_min', 'hr_25', 'hr_75', 'hr_max']
#cols = ['sport','hr_avg', 'hr_min', 'hr_05', 'hr_25', 'hr_75', 'hr_95','hr_max']
#cols = ['sport','hr_avg', 'hr_min', 'hr_25', 'hr_75', 'hr_max','spd_avg']
#cols = ['sport','hr_avg', 'hr_min', 'hr_25', 'hr_75', 'hr_max','spd_avg', 'spd_min', 'spd_25', 'spd_75', 'spd_max']

cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak']
#cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak','spd_avg']
#cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak','spd_95']
#cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak','spd_avg','spd_25','spd_75']
#cols = ['sport','hr_outof', 'hr_fatburn', 'hr_cardio', 'hr_peak','spd_avg', 'spd_min', 'spd_25', 'spd_75', 'spd_max']

df_model = df[cols]

In [14]:
# select only sports with minimal rows

count = df_model['sport'].value_counts()

#count_cond = count[count > 1].indexkn
#count_cond = count[count >= 5].inde
#count_cond = count[count >= 10].index
#count_cond = count[count >= 50].index
count_cond = count[count >= 70].index
#count_cond = count[count >= 100].index
#count_cond = count[count >= 200].index
#count_cond = count[count >= 800].index
#count_cond = count[count >= 1500].index
#count_cond = count[count >= 70000].index

count_mask = df_model['sport'].isin(count_cond)
df_model = df_model[count_mask]

In [15]:
print(df_model.shape)
df_model.head()

(166411, 5)


sport  hr_outof  hr_fatburn  hr_cardio   hr_peak
0  bike       0.0    0.013333   0.464444  0.522222
1  bike       0.0    0.020000   0.591111  0.388889
2  bike       0.0    0.057778   0.782222  0.160000
3  bike       0.0    0.037778   0.637778  0.324444
4  bike       0.0    0.011111   0.080000  0.908889

In [16]:
df_model['sport'].value_counts()

bike             91772
run              70505
walk              1539
skiing            1068
orienteering       867
skate              260
hiking             237
kayaking            92
rowing              71
Name: sport, dtype: int64

### Plot df_model

In [17]:
# order = df_model.groupby('sport')['hr_max'].median().sort_values(ascending=False).index

# plt.figure(figsize=(20,15))
# #plt.xlim(0,300)

# sns.boxplot(data=df_model,x='hr_max',y='sport',order=order);

In [18]:
# order = df_model.groupby('sport')['hr_avg'].median().sort_values(ascending=False).index

# plt.figure(figsize=(20,15))
# #plt.xlim(0,300)

# sns.boxplot(data=df_model,x='hr_avg',y='sport',order=order);

In [19]:
# order = df_model.groupby('sport')['hr_min'].median().sort_values(ascending=False).index

# plt.figure(figsize=(20,15))
# #plt.xlim(0,300)

# sns.boxplot(data=df_model,x='hr_min',y='sport',order=order);

### Create Features and Target

In [20]:
df_model.head()

sport  hr_outof  hr_fatburn  hr_cardio   hr_peak
0  bike       0.0    0.013333   0.464444  0.522222
1  bike       0.0    0.020000   0.591111  0.388889
2  bike       0.0    0.057778   0.782222  0.160000
3  bike       0.0    0.037778   0.637778  0.324444
4  bike       0.0    0.011111   0.080000  0.908889

In [21]:
# doing scaling and encoding before create X and y

cols = df_model.columns[1:]

ss = StandardScaler()
df_model[cols] = ss.fit_transform(df_model[cols])

le = LabelEncoder()
df_model['sport'] = le.fit_transform(df_model['sport'])

In [22]:
# create feature and target. next perform train_test_split

X = df_model.drop(columns='sport')
y = df_model['sport']

#le = LabelEncoder()
#y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3,stratify=y,random_state=3050)

In [23]:
# ss = StandardScaler()
# ss.fit(X_train)

# X_train = ss.transform(X_train)
# X_test = ss.transform(X_test)

In [24]:
df_model.head()

sport  hr_outof  hr_fatburn  hr_cardio   hr_peak
0      0 -0.297064   -0.789696  -0.059138  0.758145
1      0 -0.297064   -0.763360   0.383970  0.335522
2      0 -0.297064   -0.614128   1.052519 -0.389982
3      0 -0.297064   -0.693133   0.547221  0.131254
4      0 -0.297064   -0.798474  -1.404010  1.983754

### Handle Imbalanced Data

In [25]:
# check before upsample/downsample

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(116487, 4)
(49924, 4)
(116487,)
(49924,)


In [26]:
# check before upsample/downsample

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 64240, 1: 166, 2: 64, 3: 607, 4: 50, 5: 49353, 6: 182, 7: 748, 8: 1077}

In [27]:
# sm = SMOTE(random_state=3050)
# X_train, y_train = sm.fit_sample(X_train, y_train.ravel())

In [28]:
# get majority class index and row count

# sport_counts = df_model['sport'].value_counts()
# print(sport_counts)

# major_class_index = sport_counts.index[0]
# major_class_count = sport_counts.values[0]
# print(major_class_index,major_class_count)

In [29]:
# concatenate our training data back together

Xy_train = pd.concat([X_train, y_train], axis=1)

sport_counts = Xy_train['sport'].value_counts()
print(sport_counts)

print(Xy_train.shape)
Xy_train.head()

0    64240
5    49353
8     1077
7      748
3      607
6      182
1      166
2       64
4       50
Name: sport, dtype: int64
(116487, 5)


hr_outof  hr_fatburn  hr_cardio   hr_peak  sport
57703 -0.238052    1.720926  -0.471151 -0.897130      5
82346 -0.149533   -0.456117   1.106936 -0.601294      0
34004 -0.297064   -0.816031  -1.590582  2.166891      5
46482 -0.238052   -0.008418   0.889269 -0.742168      5
39385 -0.267558   -0.710690  -0.906485  1.455474      0

In [30]:
# perform upsampling and downsampling

sample_size = 13000

df_all_sample = pd.DataFrame()

sport_list = df_model['sport'].unique()
sport_list

for sport in sport_list:
    
    cond = Xy_train['sport'] == sport
    df_sport = Xy_train[cond]
    
    # perform downsampling
    if sport_counts[sport] >= sample_size:    
        print('downsampling',sport,sport_counts[sport])
        df_sample = df_sport.sample(sample_size,replace=False,random_state=3050)
        
    # perform upsampling
    # sport_counts[sport] < sample_size: 
    else:
        print('upsampling',sport,sport_counts[sport])
        df_sample = df_sport.sample(sample_size,replace=True,random_state=3050)
        
    df_all_sample = pd.concat([df_all_sample, df_sample], axis=0)
    
X_train = df_all_sample.drop(columns='sport').values
y_train = df_all_sample['sport'].values

downsampling 0 64240
downsampling 5 49353
upsampling 4 50
upsampling 3 607
upsampling 2 64
upsampling 6 182
upsampling 7 748
upsampling 8 1077
upsampling 1 166


In [31]:
# # perform upsampling for minority classes

# df_all_sample = pd.DataFrame()

# sport_list = df_model['sport'].unique()

# for sport in sport_list:
    
#     if sport != major_class_index:
        
#         cond = df_model['sport'] == sport
#         df_sport = df_model[cond]
#         #print(sport,len(df_sport))
        
#         df_sample = df_sport.sample(major_class_count,replace=True,random_state=3050)
#         df_all_sample = pd.concat([df_all_sample, df_sample], axis=0)
        
# cond = df_model['sport'] == major_class_index
# df_top = df_model[cond]
# df_all_sample = pd.concat([df_all_sample, df_top], axis=0)

# X_train = df_all_sample.drop(columns='sport').values
# y_train = df_all_sample['sport'].values

In [32]:
# check after upsample/downsample

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(117000, 4)
(49924, 4)
(117000,)
(49924,)


In [33]:
# check after upsample/downsample

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 13000,
 1: 13000,
 2: 13000,
 3: 13000,
 4: 13000,
 5: 13000,
 6: 13000,
 7: 13000,
 8: 13000}

### Logistic Regresion Model

In [34]:
# init model

logreg = LogisticRegression()

In [35]:
# perform cross validation

score = cross_val_score(logreg,X,y,cv=5)
print('score:',score.mean(),score)

# sample_size = 10000
# score: 0.9171444825052808 [0.91460533 0.91459767 0.93424281 0.89779799 0.92447862]

score: 0.7064252691013257 [0.71567626 0.70575334 0.71569364 0.69170673 0.70329637]


In [36]:
# fit model

logreg.fit(X_train, y_train)

# score model
print("train r2:",logreg.score(X_train, y_train))
print("test r2:",logreg.score(X_test, y_test))

# sample_size = 10000
#train r2: 0.40617857142857144
#test r2: 0.6280887896132905

train r2: 0.25635042735042735
test r2: 0.36561573591859625


In [37]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = logreg.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = logreg.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_) 

rows:actual columns:predicted


bike  hiking  kayaking  orienteering  rowing          run  \
bike           8186     207       323          3995            413  6924   
hiking            8       2         2             2              0     7   
kayaking          5       1         3             1              1     2   
orienteering     39       0         1           129              2    68   
rowing            3       0         1             2              2     5   
run            2214     114       156          7794            132  9563   
skate            24       0         2             9              2    17   
skiing           69       1         2            64              2    85   
walk             46       2         6             0              0    34   

               skate  skiing  walk  
bike            1797     428  5259  
hiking             4       0    46  
kayaking           4       0    11  
orienteering       7       2    12  
rowing             2       1     5  
run              296     101   782  
skate              4       0    20  
skiing            23       4    70  
walk              10       4   360

### KNN Model

In [38]:
# init model

knn = KNeighborsClassifier(n_neighbors=3)

In [39]:
# perform cross validation

score = cross_val_score(knn,X,y,cv=5)
print('score:',score.mean(),score)

score: 0.7129760109502585 [0.71195097 0.70253868 0.72296506 0.71183894 0.71558641]


In [40]:
# fit model

knn.fit(X_train, y_train)

# score model
print("train r2:",knn.score(X_train, y_train))
print("test r2:",knn.score(X_test, y_test))

# sample_size = 10000
#train r2: 0.9733142857142857
#test r2: 0.8036935840928582

train r2: 0.917017094017094
test r2: 0.5883743289800497


In [41]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = knn.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_)

rows:actual columns:predicted


bike  hiking  kayaking  orienteering  rowing           run  \
bike           16247     314       177           963            124   5603   
hiking            29       2         0             1              1      4   
kayaking          17       0         1             0              0      4   
orienteering      77       2         1            21              2    134   
rowing            12       1         0             0              1      5   
run             5117      77        97          1114            105  12853   
skate             40       0         0             0              0     21   
skiing           169       3         1            21              2     64   
walk             131      26        15             6              4     27   

               skate  skiing  walk  
bike             511    1708  1885  
hiking             3       3    28  
kayaking           0       1     5  
orienteering       4      12     7  
rowing             0       1     1  
run              267     966   556  
skate              2       7     8  
skiing            11      26    23  
walk              11      21   221

### DTC Model

In [42]:
# init model

dtc = DecisionTreeClassifier(max_depth=10,random_state=3050)

In [43]:
# perform cross validation

score = cross_val_score(dtc,X,y,cv=5)
print(score.mean(),score)

# sample_size = 10000
# 0.9436500430477148 [0.94146502 0.94489979 0.94803446 0.93923528 0.94461566]

0.7554789909901052 [0.75662441 0.74859546 0.76644332 0.75246394 0.75326783]


In [44]:
# fit model
dtc = dtc.fit(X_train,y_train)

# score model
print("train r2:",dtc.score(X_train, y_train))
print("test r2:",dtc.score(X_test, y_test))

# sample_size = 10000
# print("train r2:",dtc.score(X_train, y_train))
# print("test r2:",dtc.score(X_test, y_test))

train r2: 0.5778547008547008
test r2: 0.33018187645220737


In [45]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = dtc.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = dtc.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_)

rows:actual columns:predicted


bike  hiking  kayaking  orienteering  rowing          run  \
bike           7867    2388       642          2023           2178  2080   
hiking           11      34         2             2              0     1   
kayaking          6       3         1             0              2     1   
orienteering     41       5         4            69             12    76   
rowing            5       1         2             2              2     3   
run            2055     270       360          3641           1832  8231   
skate            18       5         3             9              5     6   
skiing           67      22         8            29             30    40   
walk             33      85        23             9             14     8   

               skate  skiing  walk  
bike            4010    4632  1712  
hiking             3       4    14  
kayaking           5       4     6  
orienteering      29      21     3  
rowing             1       3     2  
run             2546    1644   573  
skate             15      11     6  
skiing            56      48    20  
walk              45      28   217

In [46]:
# # initialize the output file object
# dot_data = StringIO() 

# # my fit DecisionTreeRegressor object here is: dtr1
# # for feature_names i put the columns of my Xr matrix
# export_graphviz(dtc, 
#                 out_file=dot_data,  
#                 filled=True, 
#                 rounded=True,
#                 special_characters=True,
#                 feature_names=df_model[features].columns
#                )  

# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

### RFC Model

In [47]:
# init model

rfc = RandomForestClassifier(n_estimators=10,max_depth=10,n_jobs=2, random_state=3050)

In [48]:
# perform cross validation

score = cross_val_score(rfc,X,y,cv=5)
print(score.mean(),score)

# sample_size = 10000
# 0.9467134535825069 [0.94526366 0.94657493 0.95105606 0.94357348 0.94709913]

0.7573058006837113 [0.75785616 0.75066847 0.77230251 0.7476863  0.75801557]


In [49]:
# fit model
rfc = rfc.fit(X_train,y_train)

# score model
print("train r2:",rfc.score(X_train, y_train))
print("test r2:",rfc.score(X_test, y_test))

# sample_size = 10000
# train r2: 0.8223142857142857
# test r2: 0.7444007897728406

train r2: 0.6981709401709402
test r2: 0.4575354538899127


In [50]:
# confusion matrix
print('rows:actual columns:predicted')

y_pred = rfc.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(data=cm, columns=le.classes_, index=le.classes_)

# y_pred = rfc.predict_proba(X_test)
# pd.DataFrame(y_pred,columns=le.classes_)

rows:actual columns:predicted


bike  hiking  kayaking  orienteering  rowing           run  \
bike           11691    2257       511          1625            838   3563   
hiking            10      28         1             1              0      2   
kayaking           9       3         2             1              2      1   
orienteering      49       7         5            73              6     80   
rowing             7       2         1             0              2      5   
run             3240     321       200          3725            417  10752   
skate             22       8         0             4              2     14   
skiing            93      26         6            24              7     68   
walk              52      79        17             5              3     22   

               skate  skiing  walk  
bike            2668    2790  1589  
hiking             2       4    23  
kayaking           4       1     5  
orienteering      22      12     6  
rowing             1       1     2  
run              944    1086   467  
skate             16       9     3  
skiing            39      41    16  
walk              34      13   237

### GridSearch Model

In [51]:
# init models

estimators = {
    'lr': LogisticRegression(),
    'knn': KNeighborsClassifier(),
    'dtc': DecisionTreeClassifier(),
    'rfc': RandomForestClassifier(),
    'abc': AdaBoostClassifier(),
    'gbc': GradientBoostingClassifier()
}.items()

In [52]:
# init model parameters

params = {
    'lr': {

    },
    'knn': {
        'knn__n_neighbors': [3,5,7,9]
    },

    'dtc': {
        'dtc__max_features': ['auto', 'log2', None],
        'dtc__max_depth': [None, 5, 10, 15],
        'dtc__min_samples_split': np.linspace(0.1, 0.5, 5)
    },
    'rfc': {
        'rfc__n_estimators': [10, 20, 50, 100],
        'rfc__max_depth': [None, 5, 10, 15],
        'rfc__max_features': ['auto', 'log2', None],
        'rfc__min_samples_split': np.linspace(0.1, 0.5, 5)
    },
    'abc': {

    }, 
    'gbc': {

    }
}

In [53]:
models = []
parameters = []
best_score = []
roc_auc = []

for k,v in estimators:
    
    pipe = Pipeline([
        ('sc', StandardScaler()),
        (k,v)])
    
    gridsearch = GridSearchCV(
        estimator=pipe,
        param_grid=params[k],
        verbose=1,
        cv= 5,
        n_jobs=-1,
        return_train_score= True
        #scoring = 'roc_auc'
    )

    gridsearch.fit(X_train, y_train)
    
    model = gridsearch.best_estimator_
    cv_score = gridsearch.cv_results_
    best_params = gridsearch.best_params_

    # predict y
    #y_pred = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    
    # print results
    print("Model: ", k)
    print("Best parameters:", best_params)
    print("Best score:", gridsearch.best_score_)
    #print("AUC/ROC test:", roc_auc_score(y_test,y_pred))
    display(pd.DataFrame(cv_score, columns = cv_score.keys()))    
    
    # append info to list
    models.append(k)
    best_score.append(gridsearch.best_score_)
    parameters.append(best_params)
    #roc_auc.append(roc_auc_score(y_test,y_pred))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s finished


Model:  lr
Best parameters: {}
Best score: 0.2556752136752137


mean_fit_time  std_fit_time  mean_score_time  std_score_time params  \
0       1.362949      0.139064         0.012499         0.00625     {}   

   split0_test_score  split1_test_score  split2_test_score  split3_test_score  \
0           0.257094           0.255299           0.255684           0.257778   

   split4_test_score  mean_test_score  std_test_score  rank_test_score  \
0           0.252521         0.255675        0.001818                1   

   split0_train_score  split1_train_score  split2_train_score  \
0            0.256165            0.255919            0.255278   

   split3_train_score  split4_train_score  mean_train_score  std_train_score  
0            0.256453            0.254145          0.255592         0.000821

Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

mean_fit_time  std_fit_time  mean_score_time  std_score_time params  \
0       6.339965      0.585224         0.787314        0.145068     {}   

   split0_test_score  split1_test_score  split2_test_score  split3_test_score  \
0           0.307949           0.297991           0.313504           0.305897   

   split4_test_score  mean_test_score  std_test_score  rank_test_score  \
0            0.30312         0.305692         0.00514                1   

   split0_train_score  split1_train_score  split2_train_score  \
0            0.305342            0.300919            0.310342   

   split3_train_score  split4_train_score  mean_train_score  std_train_score  
0             0.30188            0.308494          0.305395         0.003646

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.8min finished


Model:  gbc
Best parameters: {}
Best score: 0.6024957264957265


mean_fit_time  std_fit_time  mean_score_time  std_score_time params  \
0      136.19757     32.452758         0.509264        0.107125     {}   

   split0_test_score  split1_test_score  split2_test_score  split3_test_score  \
0           0.608932            0.60235            0.60453           0.597179   

   split4_test_score  mean_test_score  std_test_score  rank_test_score  \
0           0.599487         0.602496        0.004072                1   

   split0_train_score  split1_train_score  split2_train_score  \
0            0.614263            0.609605            0.609081   

   split3_train_score  split4_train_score  mean_train_score  std_train_score  
0            0.610256            0.605908          0.609823         0.002677

In [54]:
# sample_size = 10000

# Model:  lr
# Best parameters: {}
# Best score: 0.40513571428571427

# Model:  knn
# Best parameters: {'knn__n_neighbors': 3}
# Best score: 0.9692571428571428

# Model:  dtc
# Best parameters: {'dtc__max_depth': None, 'dtc__max_features': None, 'dtc__min_samples_split': 0.1}
# Best score: 0.5181714285714286

# Model:  rfc
# Best parameters: {'rfc__max_depth': 15, 'rfc__max_features': 'auto', 'rfc__min_samples_split': 0.1, 'rfc__n_estimators': 100}
# Best score: 0.4909071428571429

# Model:  abc
# Best parameters: {}
# Best score: 0.23792857142857143

# Model:  gbc
# Best parameters: {}
# Best score: 0.8113571428571429